In [2]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import json
import numpy as np
import os

/home/codespace/.cache/pypoetry/virtualenvs/finetune-f9B3-yHi-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Define the dataset class
class ImageDataset(Dataset):
    def __init__(self, json_file, transform=None):
        with open(json_file, 'r') as f:
            self.data = json.load(f)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img_path = item['image_path']
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, idx

# Load the pre-trained ResNet50 model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Identity()  # Modify the model to return embeddings from the penultimate layer
model = model.to(device)
model.eval()

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def compute_embeddings_and_update_json(json_file):
    dataset = ImageDataset(json_file, transform=transform)
    loader = DataLoader(dataset, batch_size=1, shuffle=False)

    embeddings = []
    with torch.no_grad():
        for images, idxs in loader:
            images = images.to(device)
            output = model(images).cpu().numpy().flatten()
            embeddings.append((idxs.item(), output))

    with open(json_file, 'r+') as f:
        data = json.load(f)
        for idx, emb in embeddings:
            data[idx]['embedding'] = emb.tolist()
        f.seek(0)
        json.dump(data, f, indent=4)
        f.truncate()

# Paths to the JSON files
json_files = [
    '/workspaces/finetune/AFINAL/resnet/image_paths.json',
]

# Compute embeddings and update JSON files
for json_file in json_files:
    compute_embeddings_and_update_json(json_file)


/home/codespace/.cache/pypoetry/virtualenvs/finetune-f9B3-yHi-py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/codespace/.cache/pypoetry/virtualenvs/finetune-f9B3-yHi-py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
class CustomDataset(Dataset):
    def __init__(self, json_file, transform=None):
        with open(json_file, 'r') as f:
            self.data = json.load(f)
        self.transform = transform
        self.classes = list(set(item['class'] for item in self.data))
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.idx_to_class = {idx: cls_name for cls_name, idx in self.class_to_idx.items()}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]['image_path']
        label = self.class_to_idx[self.data[idx]['class']]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
def embed_images_and_update_json(model, dataset, device, json_file):
    loader = DataLoader(dataset, batch_size=1, shuffle=False)
    model.eval()
    embeddings = []

    with torch.no_grad():
        for images, idxs in loader:
            images = images.to(device)
            embeddings_output = model(images, return_embedding=True).cpu().numpy().flatten()
            embeddings.append((idxs.item(), embeddings_output))

    # Update JSON file with embeddings
    with open(json_file, 'r+') as f:
        data = json.load(f)
        for idx, emb in embeddings:
            data[idx]['embedding'] = emb.tolist()
        f.seek(0)
        json.dump(data, f, indent=4)
        f.truncate()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True)
model.fc = torch.nn.Identity()  # Modify the model to return embeddings from the penultimate layer
model = model.to(device)
model.eval()

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Process training and validation data
#train_json = '/workspaces/finetune/AFINAL/resnet/output/train_data.json'
#val_json = '/workspaces/finetune/AFINAL/resnet/output/val_data.json'
#test_json = '/workspaces/finetune/AFINAL/resnet/output/test_data.json'
original = '/workspaces/finetune/AFINAL/resnet/image_paths_base.json'

#train_dataset = CustomDataset(json_file=train_json, transform=transform)
#val_dataset = CustomDataset(json_file=val_json, transform=transform)
#test_dataset = CustomDataset(json_file=test_json, transform=transform)
original_dataset = CustomDataset(json_file=original, transform=transform)

#embed_images_and_update_json(model, train_dataset, device, train_json)
#embed_images_and_update_json(model, val_dataset, device, val_json)
#embed_images_and_update_json(model, test_dataset, device, test_json)
embed_images_and_update_json(model, original_dataset, device, original)


TypeError: ResNet.forward() got an unexpected keyword argument 'return_embedding'

In [5]:
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Function to load data from JSON file
def load_data(filepath):
    with open(filepath, 'r') as file:
        return json.load(file)

# Function to compute Top-K accuracy
def compute_top_k_accuracy(query_data, gallery_data, k):
    total_queries = len(query_data)
    top_k_hits = 0

    # Preparing the gallery embeddings and classes
    gallery_embeddings = np.array([item['embedding'] for item in gallery_data])
    gallery_classes = np.array([item['class'] for item in gallery_data])

    # Loop through each query image
    for query in query_data:
        query_embedding = np.array([query['embedding']])
        query_class = query['class']

        # Compute cosine similarities between this query and all gallery images
        similarities = cosine_similarity(query_embedding, gallery_embeddings)[0]

        # Get indices of the top K most similar images
        top_k_indices = np.argsort(similarities)[-k:]

        # Check if the correct class is within the top K similar images
        if query_class in gallery_classes[top_k_indices]:
            top_k_hits += 1

    # Calculate average Top-K accuracy
    top_k_accuracy = top_k_hits / total_queries
    return top_k_accuracy

# Load datasets
test_data = load_data('/workspaces/finetune/AFINAL/resnet/output_base/test_data.json')
val_data = load_data('/workspaces/finetune/AFINAL/resnet/output_base/val_data.json')
train_data = load_data('/workspaces/finetune/AFINAL/resnet/output_base/train_data.json')

# Combine validation and training data into one gallery set
gallery_data = val_data + train_data

# Set the value of K  # Adjust based on your specific requirements

# Compute the Top-K accuracy
for k in range(11):
    average_top_k_accuracy = compute_top_k_accuracy(test_data, gallery_data, k)
    print(f"Average Top-{k} Accuracy: {average_top_k_accuracy:.2f}")



Average Top-0 Accuracy: 1.00
Average Top-1 Accuracy: 0.96
Average Top-2 Accuracy: 0.97
Average Top-3 Accuracy: 0.97
Average Top-4 Accuracy: 0.97
Average Top-5 Accuracy: 0.98
Average Top-6 Accuracy: 0.98
Average Top-7 Accuracy: 0.98
Average Top-8 Accuracy: 0.98
Average Top-9 Accuracy: 0.98
Average Top-10 Accuracy: 0.98


In [11]:
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import random

# Function to load data from JSON file
def load_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    return data

# Function to randomly select query images
def select_query_images(data, num_queries=50):
    # Filter records with "image_paths" length over 2
    eligible_data = [record for record in data if len(record['image_paths']) > 2]
    # Ensure each class is represented only once
    class_seen = set()
    filtered_data = []
    for record in eligible_data:
        if record['class'] not in class_seen:
            filtered_data.append(record)
            class_seen.add(record['class'])
    # Randomly select 50 records from the filtered data
    query_images = random.sample(filtered_data, min(num_queries, len(filtered_data)))
    return query_images

# Function to compute top K accuracy
def compute_top_k_accuracy(query_images, all_data, k):
    gallery_images = [item for item in all_data if item not in query_images]
    gallery_embeddings = np.array([item['embedding'] for item in gallery_images])
    gallery_classes = [item['class'] for item in gallery_images]

    total_queries = len(query_images)
    top_k_hits = 0

    for query in query_images:
        query_embedding = np.array([query['embedding']])
        query_class = query['class']
        similarities = cosine_similarity(query_embedding, gallery_embeddings)[0]
        top_k_indices = np.argsort(similarities)[-k:]
        if query_class in np.array(gallery_classes)[top_k_indices]:
            top_k_hits += 1

    return top_k_hits / total_queries

# Load the data
data = load_data('/workspaces/finetune/AFINAL/resnet/image_paths_with_base_embedding.json')

# Select query images
query_images = select_query_images(data)

# Compute the Top-K accuracy
for k in range(1,11):
    top_k_accuracy = compute_top_k_accuracy(query_images, data, k)  # Adjust K value as needed
    print(f"Average Top-{k} Accuracy: {top_k_accuracy:.4f}")


Average Top-1 Accuracy: 0.3200
Average Top-2 Accuracy: 0.3600
Average Top-3 Accuracy: 0.3600
Average Top-4 Accuracy: 0.3600
Average Top-5 Accuracy: 0.3600
Average Top-6 Accuracy: 0.3800
Average Top-7 Accuracy: 0.4000
Average Top-8 Accuracy: 0.4000
Average Top-9 Accuracy: 0.4000
Average Top-10 Accuracy: 0.4000


In [8]:
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Function to load data from JSON file
def load_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    return data

# Load the datasets
query_data = load_data('/workspaces/finetune/AFINAL/resnet/output_base/test_data.json')
train_data = load_data('/workspaces/finetune/AFINAL/resnet/output_base/train_data.json')
val_data = load_data('/workspaces/finetune/AFINAL/resnet/output_base/val_data.json')
image_paths_data = load_data('/workspaces/finetune/AFINAL/resnet/image_paths_base.json')

# Combine train and val data for the gallery
gallery_data = train_data + val_data

# Function to find image paths for exclusion
def find_exclusion_paths(image_path):
    for record in image_paths_data:
        if image_path in record['image_paths']:
            return set(record['image_paths'])
    return set()

# Function to compute top K accuracy
def compute_top_k_accuracy(query_data, gallery_data, k):
    top_k_hits = 0
    total_queries = len(query_data)

    for query in query_data:
        exclusion_paths = find_exclusion_paths(query['image_path'])
        # Filter gallery images: exclude the current query and its transformations
        filtered_gallery = [item for item in gallery_data if item['image_path'] not in exclusion_paths]

        # Prepare data for similarity computation
        query_embedding = np.array([query['embedding']])
        gallery_embeddings = np.array([item['embedding'] for item in filtered_gallery])
        gallery_classes = [item['class'] for item in filtered_gallery]

        # Compute cosine similarities
        similarities = cosine_similarity(query_embedding, gallery_embeddings)[0]
        top_k_indices = np.argsort(similarities)[-k:]
        top_k_classes = np.array(gallery_classes)[top_k_indices]

        # Check if the correct class is within the top K similar images
        if query['class'] in top_k_classes:
            top_k_hits += 1

    return top_k_hits / total_queries

# Calculate average Top-K accuracy
k_values = [1, 5, 10]  # Example K values
for k in k_values:
    average_top_k_accuracy = compute_top_k_accuracy(query_data, gallery_data, k)
    print(f"Average Top-{k} Accuracy: {average_top_k_accuracy:.4f}")


Average Top-1 Accuracy: 0.4168
Average Top-5 Accuracy: 0.5043
Average Top-10 Accuracy: 0.5409
